In [59]:
import torchvision
from torchvision.datasets import CIFAR10
import torch
torch.manual_seed(0)


In [60]:
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
class MyDataset(Dataset):
   def __init__(self, np_file_paths):
        #self.data = torch.FloatTensor(data.values.astype('float'))
        self.files = os.listdir(np_file_paths)
        self.np_file_paths = np_file_paths
   def __getitem__(self, index):
     x = np.load("{}/{}".format(self.np_file_paths, self.files[index]))
     x = torch.from_numpy(x).float()
     label = torch.tensor(float(self.files[index].split("_")[4]))
     return x,label
   def __len__(self):
        return len(self.files)

        
#train_dataset = '/content/gdrive/"My Drive"/trainprob'
#test_dataset = '/content/gdrive/"My Drive"/testprob'


In [61]:
train_dataset =  MyDataset('/content/drive/MyDrive/trainprob/')
test_dataset = MyDataset('/content/drive/MyDrive/testprob/')
val_dataset = MyDataset('/content/drive/MyDrive/valueprob/')

In [62]:
train_dataset[12]

(tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
       

In [63]:
print(len(val_dataset))

313


In [64]:

trainloader = DataLoader(train_dataset, batch_size = 64)
valloader = DataLoader(val_dataset, batch_size = 64)
testloader = DataLoader(test_dataset, batch_size = 64)

In [65]:
#у меня 22, а не 20
#сетка имеет один и тот же размер везде, будет плохо работать, количество входных выходных каналов #16 32 64  и обратно
import torch
import torch.nn as nn
class _Model(torch.nn.Module):
    """A neural network model to predict phylogenetic trees."""

    def __init__(self):
        """Create a neural network model."""
        super().__init__()
        self.conv = torch.nn.Sequential(
            torch.nn.Conv1d(88, 88, 1, groups=22),
            torch.nn.BatchNorm1d(88),
            torch.nn.ReLU(),
            torch.nn.Conv1d(88, 32, 1),
            torch.nn.BatchNorm1d(32),
            torch.nn.ReLU(),
            _ResidueModule(32),
            _ResidueModule(32),
            torch.nn.AvgPool1d(2),
            _ResidueModule(32),
            _ResidueModule(32),
            torch.nn.AvgPool1d(2),
            _ResidueModule(32),
            _ResidueModule(32),
            torch.nn.AvgPool1d(2),
            _ResidueModule(32),
            _ResidueModule(32),
            torch.nn.AdaptiveAvgPool1d(1),
            #torch.nn.Softmax(dim =-1)
        )
        self.classifier = torch.nn.Linear(32, 1)

    def forward(self, x):
        x = torch.reshape(x.transpose(2,3),(x.size()[0], 88, -1))
        #x = x.view(x.size()[0], 88, -1)
        x = self.conv(x).squeeze(dim=2)
        return self.classifier(x)
class _ResidueModule(torch.nn.Module):

    def __init__(self, channel_count):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Conv1d(channel_count, channel_count, 1),
            torch.nn.BatchNorm1d(channel_count),
            torch.nn.ReLU(),
            torch.nn.Conv1d(channel_count, channel_count, 1),
            torch.nn.BatchNorm1d(channel_count),
            torch.nn.ReLU()
            
        )

    def forward(self, x):
        return x + self.layers(x)

In [66]:
##непонятно, как писать train

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # change run time to gpu to fast training

model = _Model().to(device)
#criterion = nn.CrossEntropyLoss()
#criterion = nn.BCELoss()
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

num_epochs = 15
loss_hist = [] # for plotting
for epoch in range(num_epochs):
    hist_loss = 0
    for _, batch in enumerate(trainloader, 0): # get batch
        # parse batch 
        imgs, labels = batch
        imgs, labels = imgs.to(device), labels.to(device)
        # sets the gradients of all optimized tensors to zero.
        optimizer.zero_grad() 
        # get outputs
        Y_pred = model(imgs)
        Y_pred = Y_pred.squeeze(axis=1)
        #print(Y_pred.shape)
        #print(labels.shape)
        # calculate loss
        loss = criterion(Y_pred, labels)
        # calculate gradients
        loss.backward() 
        # performs a single optimization step (parameter update)
        optimizer.step()
        hist_loss += loss.item()
    loss_hist.append(hist_loss /len(trainloader))
    print(f"Epoch={epoch} loss={loss_hist[epoch]:.4f}")

Epoch=0 loss=0.8928
Epoch=1 loss=0.7227
Epoch=2 loss=0.6568
Epoch=3 loss=0.6366
Epoch=4 loss=0.6279
Epoch=5 loss=0.6210
Epoch=6 loss=0.6138
Epoch=7 loss=0.6060
Epoch=8 loss=0.5973
Epoch=9 loss=0.5876
Epoch=10 loss=0.5775
Epoch=11 loss=0.5663
Epoch=12 loss=0.5544
Epoch=13 loss=0.5421
Epoch=14 loss=0.5292


In [106]:
def calaculate_accuracy(model, data_loader):
    correct, total = 0, 0 
    with torch.no_grad(): 
        for batch in data_loader: # get batch
            imgs, labels = batch # parse batch
            imgs, labels = imgs.to(device), labels.to(device)
            #print(imgs.shape)
            Y_pred = model.forward(imgs) # get output
            #print(Y_pred)
            predicted=torch.sigmoid(Y_pred).squeeze()
            print(predicted)
            total += labels.shape[0] # all examples
            #print(labels.shape[0])
            #print(predicted.shape)
            correct += ((predicted>0.5) == labels).sum().item() 
           # print(total,correct)
    return correct / total 

In [107]:
acc_train = round(calaculate_accuracy(model, trainloader), 3)
print(f"Accuracy train = {acc_train}")
acc_test = round(calaculate_accuracy(model, testloader), 3)
print(f"Accuracy test = {acc_test}")

tensor([0.2238, 0.2219, 0.3411, 0.4204, 0.2512, 0.1822, 0.3726, 0.2012, 0.2400,
        0.6065, 0.1734, 0.2041, 0.4255, 0.6738, 0.3161, 0.1726, 0.1619, 0.2882,
        0.2502, 0.1312, 0.1918, 0.1913, 0.6996, 0.2309, 0.5041, 0.3414, 0.4505,
        0.5846, 0.5131, 0.4105, 0.5598, 0.3692, 0.3191, 0.1617, 0.1580, 0.5538,
        0.2123, 0.3680, 0.4902, 0.3623, 0.7582, 0.3211, 0.3052, 0.4375, 0.3524,
        0.2073, 0.3258, 0.3496, 0.5703, 0.1504, 0.2527, 0.2714, 0.1066, 0.0935,
        0.2474, 0.4835, 0.4340, 0.2759, 0.6166, 0.3076, 0.5010, 0.4293, 0.7255,
        0.4126])
tensor([0.1961, 0.5079, 0.6540, 0.7337, 0.2261, 0.4966, 0.5218, 0.2236, 0.3873,
        0.1212, 0.2294, 0.6321, 0.2182, 0.4579, 0.3277, 0.6266, 0.4668, 0.3550,
        0.1493, 0.5770, 0.2676, 0.5048, 0.4274, 0.2978, 0.1480, 0.5183, 0.1943,
        0.0841, 0.4552, 0.2709, 0.7293, 0.4635, 0.2109, 0.1536, 0.3523, 0.4530,
        0.2438, 0.0701, 0.2212, 0.5568, 0.2138, 0.3141, 0.4147, 0.4779, 0.7040,
        0.2200, 0.1886,